### Installation Test

Test the required packages are correctly installed.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

### ODEs

$\frac{dx(t)}{dt} = -\nabla V(x(t))$


Task:

    1. Observe the solution to the ODE with different initial states.
    
    2. What can be said about the dynamics?
    
    3. Take a look at the link below to see how is the ODE numerically solved (what is the numerical scheme).     
         
link: 
    https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html#scipy.integrate.solve_ivp


In [ ]:
from scipy.integrate import solve_ivp

def V(x):
    return 0.25 * (x[0]**2 - 1.0)**2 + 0.5 * x[1]**2 

def gradV(x):    
    return np.array([x[0] * (x[0]**2 - 1.0), x[1]]) 

def fun(t, x):
    return -1.0 * gradV(x)

# initial state 
x0 = np.array([0.5, 1.0])
# terminal time
T = 3.0        
# step-size
dt = 0.1
t_eval = np.arange(0, T, dt)
    
sol = solve_ivp(fun, [0, T], x0, t_eval=t_eval)

x = np.arange(-1.5, 1.5, 0.05)
y = np.arange(-1.0, 1.0, 0.05)
X, Y = np.meshgrid(x, y)

plt.figure(figsize = (10, 6))

# contour line of the potential
plt.subplot(221)
V_vec = V([X,Y])
CS = plt.contour(X, Y, V_vec, levels=[0,0.1,0.2,0.3,0.4,0.5])
plt.xlabel('x')
plt.ylabel('y')

# x component of the ODE solution
plt.subplot(222)
plt.plot(sol.t, sol.y[0,:])
plt.xlabel('t')
plt.ylabel('x')
tmp = plt.yticks([-1,-0.5,0,0.5,1])

# y component of the ODE solution
plt.subplot(223)
plt.plot(sol.t, sol.y[1,:])
plt.xlabel('t')
plt.ylabel('y')
tmp = plt.yticks([-1,-0.5,0,0.5,1])

# potential 
plt.subplot(224)
v_sol = V(sol.y)
plt.plot(sol.t, v_sol)
plt.xlabel('t')
plt.ylabel('V')

### Hamiltonian ODE

$\begin{equation}
\begin{aligned}
\frac{dq(t)}{dt} =& \nabla_p H(q(t), p(t)) \\
\frac{dp(t)}{dt} =& -\nabla_q H(q(t), p(t)) \\
\end{aligned}
\end{equation}
$

Task:

    . Can you write down the ODE?
    
    . Vary the initial state and see how the solution changes?
    
    . Increase the simulation time to $T=300$. What changes? 
    

In [ ]:
def H(x):
    # x=(q,p)
    return 0.25 * (x[0]**2 - 1.0)**2 + 0.5 * x[1]**2 

def gradH(x):    
    return np.array([x[0] * (x[0]**2 - 1.0), x[1]]) 

def fun(t, x):
    dH = gradH(x)
    return np.array([dH[1], -dH[0]])     

# initial state 
x0 = np.array([0.5, 1.0])
# terminal time
T = 30.0        
# step-size
dt = 0.01
t_eval = np.arange(0, T, dt)
    
sol = solve_ivp(fun, [0, T], x0, t_eval=t_eval)

x = np.arange(-1.5, 1.5, 0.05)
y = np.arange(-1.0, 1.0, 0.05)
X, Y = np.meshgrid(x, y)

plt.figure(figsize = (10, 6))

# contour line of the potential
plt.subplot(221)
CS = plt.contour(X, Y, V_vec, levels=[0,0.1,0.2,0.3,0.4,0.5])
plt.scatter(sol.y[0,:], sol.y[1,:], s=1, c='black', marker='o')
plt.xlabel('q')
plt.ylabel('p')

# x component of the ODE solution
plt.subplot(222)
plt.plot(sol.t, sol.y[0,:])
plt.xlabel('t')
plt.ylabel('q')
tmp = plt.yticks([-1,-0.5,0,0.5,1])

# p component of the ODE solution
plt.subplot(223)
plt.plot(sol.t, sol.y[1,:])
plt.xlabel('t')
plt.ylabel('p')
tmp = plt.yticks([-1,-0.5,0,0.5,1])

# H
plt.subplot(224)
v_sol = H(sol.y)
plt.plot(sol.t, v_sol)
plt.xlabel('t')
plt.ylabel('H')
plt.ylim([0,1])

### Solve the same Halmitonian ODE, but with a semi-implicit Euler scheme 

Task: 

    . Compare the implementation below and the scheme on wiki page [1]. Can you write down the numerical scheme?
    
    . Increase simulation time to $T=300$ and see what has changed?
    
    
1. Semi-implicit Euler scheme:    

    https://en.wikipedia.org/wiki/Semi-implicit_Euler_method    

In [ ]:
def H(x):
    # x=(q,p)
    return 0.25 * (x[0]**2 - 1.0)**2 + 0.5 * x[1]**2 

def dq_H(q):    
    return np.array(q * (q**2 - 1.0)) 

def dp_H(p):    
    return np.array(p) 

def solve_ivp_semi_implicit_euler(x0, t_eval):
    t_prev = t_eval[0]
    y = np.zeros((2,t_eval.size))
    y[:,0] = x0
    for idx, t in enumerate(t_eval[1:]):
        dt = t-t_prev
        y[0,idx+1] = y[0,idx] + dp_H(y[1,idx]) * dt
        y[1,idx+1] = y[1,idx] - dq_H(y[0,idx+1]) * dt        
        t_prev = t
    return y
    
# initial state 
x0 = np.array([0.5, 1.0])
# terminal time
T = 30.0        
# step-size
dt = 0.01
t_eval = np.arange(0, T, dt)
sol = solve_ivp_semi_implicit_euler(x0, t_eval=t_eval)

x = np.arange(-1.5, 1.5, 0.05)
y = np.arange(-1.0, 1.0, 0.05)
X, Y = np.meshgrid(x, y)

plt.figure(figsize = (10, 6))

# contour line of the potential
plt.subplot(221)
CS = plt.contour(X, Y, V_vec, levels=[0,0.1,0.2,0.3,0.4,0.5])
plt.scatter(sol[0,:], sol[1,:], s=1, c='black', marker='o')
plt.xlabel('q')
plt.ylabel('p')

# q component of the ODE solution
plt.subplot(222)
plt.plot(t_eval, sol[0,:])
plt.xlabel('t')
plt.ylabel('q')
tmp = plt.yticks([-1,-0.5,0,0.5,1])

# p component of the ODE solution
plt.subplot(223)
plt.plot(t_eval, sol[1,:])
plt.xlabel('t')
plt.ylabel('p')
tmp = plt.yticks([-1,-0.5,0,0.5,1])

# H
plt.subplot(224)
v_sol = H(sol)
plt.plot(t_eval, v_sol)
plt.xlabel('t')
plt.ylabel('H')
plt.ylim([0,1])

### Gaussian random variables

Task:
1. plot the histogram for N=500, 1000, 5000, 20000  

In [ ]:
rng = np.random.default_rng(1234)

mean, sigma = 0, 1.0
N = 500
s = rng.normal(mean, sigma, N)

count, bins, _ = plt.hist(s, 100, density=True)
plt.plot(bins, 1/(np.sqrt(2 * np.pi * sigma**2)) * np.exp(-(bins-mean)**2 / (2*sigma**2)), linewidth=2, color='r')
plt.show()

### Task: 

Can you generate Gaussian random varaibles in $\mathbb{R}^d$? Try to write the code in the cell below

### Simulate Brownain motion 

Tasks:
1. run the code 
2. test different step-size, by choosing N = 10,100,1000 
3. test different seeds
4. compute $\mathbb{E}(X[1])$ for m=5, 500, 50000

In [ ]:
# total time
T = 1
N = 1000
# step-size
dt = T/N
# no. of trajectories
m = 5

seed = 1234

rng = np.random.default_rng(seed)

n_save = 5
# only save 
x_saved = np.zeros((n_save, N+1))
x = np.zeros(m)

for n in range(N):
    dx = rng.normal(0, np.sqrt(dt), m)
    x = x + dx
    x_saved[:,n+1] = x[:n_save]

t_vec = np.linspace(0, T, N+1)

for i in range(n_save):
    plt.plot(t_vec, x_saved[i,:])

# estimate the mean of Brownian motion at time T   
print ("estimated mean at time T=%.1f using %d trajectories: %.3e" % (T,m, np.mean(x)) )  

### Task: 

Can you simulate Brownian motion in $\mathbb{R}^2$? Write the code in the cell below and plot the trajectory.